In [116]:
import pandas as pd
import requests

import plotly.express as px           # for the visualisation
import plotly.graph_objects as go
import plotly

In [117]:
# define the key
key = "cc6b6ac1327c0ae1ed73c16520864279"

# specify the city
city = input("Voer een stad in").lower().capitalize().strip()

In [118]:
# define the endpoint
endpoint = f"https://api.openweathermap.org/data/2.5/forecast?q={city}&appid={key}&units=metric"

In [119]:
# send the get request to the endpoint
response = requests.get(endpoint)

In [120]:
# get the data
data = response.json()

In [121]:
# show the keys of the data (it happens to be a dictionary)
data.keys()

dict_keys(['cod', 'message', 'cnt', 'list', 'city'])

In [122]:
data['list'][0]

{'dt': 1710860400,
 'main': {'temp': 15.51,
  'feels_like': 14.76,
  'temp_min': 13.96,
  'temp_max': 15.51,
  'pressure': 1018,
  'sea_level': 1018,
  'grnd_level': 1014,
  'humidity': 63,
  'temp_kf': 1.55},
 'weather': [{'id': 500,
   'main': 'Rain',
   'description': 'light rain',
   'icon': '10d'}],
 'clouds': {'all': 100},
 'wind': {'speed': 3.33, 'deg': 204, 'gust': 5.15},
 'visibility': 10000,
 'pop': 0.2,
 'rain': {'3h': 0.15},
 'sys': {'pod': 'd'},
 'dt_txt': '2024-03-19 15:00:00'}

In [123]:
# create a list for the temperatures
weather_temp = []
weather_feels_like = []
weather_rain_mm = []
weather_description = []
weather_dt_txt = []


# iterate through the list with predictions
for prediction in data['list']:
    weather_temp.append(prediction['main']['temp'])
    weather_feels_like.append(prediction['main']['feels_like'])
    try: # if it does not rain, open weather map does not provide the key
        weather_rain_mm.append(prediction['rain']['3h'])
    except KeyError:
        weather_rain_mm.append(0)
    weather_description.append(prediction['weather'][0]['description'])
    weather_dt_txt.append(prediction['dt_txt'])


In [124]:
# create a DataFrame from the seperate lists
df = pd.DataFrame({
    'datum': weather_dt_txt,
    'beschrijving': weather_description,
    'temperatuur': weather_temp,
    'gevoelstemperatuur': weather_feels_like,
    'regen': weather_rain_mm
})

# show the first rows of the data frame
df.head()

,datum,beschrijving,temperatuur,gevoelstemperatuur,regen
0,2024-03-19 15:00:00,light rain,15.51,14.76,0.15
1,2024-03-19 18:00:00,light rain,14.13,13.40,0.12
2,2024-03-19 21:00:00,overcast clouds,12.98,12.24,0.00
3,2024-03-20 00:00:00,overcast clouds,10.87,10.15,0.00
4,2024-03-20 03:00:00,overcast clouds,11.13,10.39,0.00


## Visualize the data

In [125]:
# create the figure
fig = go.Figure()

In [126]:
# add the temperature
fig.add_trace(
    go.Scatter(
        x=df['datum'],
        y=df['temperatuur'],
        name='Temperatuur',
        line=dict(color='royalblue', width=6)
    )
    )

In [127]:
# add the temperature
fig.add_trace(
    go.Scatter(
        x=df['datum'],
        y=df['gevoelstemperatuur'],
        name='Gevoelstemperatuur',
        line=dict(color='orange', width=3, dash='dot')
    )
    )

In [128]:
# add the rain
fig.add_trace(
    go.Bar(
        x=df['datum'],
        y=df['regen'],
        name='Regen',
        marker=dict(
                     color='rgb(158,202,225)',
                     line=dict(
                         color='rgb(8,48,107)',
                         width=1.5),
                       ),
                       opacity=0.6))

In [129]:
# add titles and layouts
fig.update_layout(
    title = f"Weather prediction for {city}",
    yaxis_title="Temperture in (C)"
)


In [131]:
# specify the filename 
filename = f"data/weather_prediction_{city}.html"

plotly.offline.plot(fig, filename, auto_open=False)

fig.write_html(filename)

'temp-plot.html'